In [1]:
import requests
import re
import mysql.connector
from dotenv import load_dotenv

load_dotenv()

user_name = os.getenv("BFL_ID")
user_pasw = os.getenv("BFL_PW")

r = requests.get('http://192.168.66.5/Status_Wireless.asp', auth=(user_name, user_pasw))


ModuleNotFoundError: No module named 'mysql'

In [14]:
def elimrep(a):
    b = [a.count(x) for x in a]
    c = [x[0] for x in zip(a, b) if x[1] == 1]
    return c

data = r.text
p = re.compile(r'(?:[0-9a-fA-F]:?){12}')

macs = []
found = re.findall(p, data)
for a in found:
    macs.append(a)

# las repetidas son la mac del router
macs = elimrep(macs)
macs

['BC:FF:EB:2C:E8:FC',
 '9C:A9:E4:F5:D4:89',
 'A8:C8:3A:7B:E5:B8',
 '9C:2A:83:75:E8:79',
 'B8:AE:ED:40:60:9B',
 '50:C8:E5:C7:34:8B',
 '00:90:4C:8D:5E:4D']

In [15]:
sql_id = os.getenv("SQL_ID")
sql_pw = os.getenv("SQL_PW")

cnx = mysql.connector.connect(user=sql_id, password=sql_pw, host='127.0.0.1', database='clientes')
cursor = cnx.cursor()

In [16]:
for l in macs:
    query = ("select datos_per.coduser, datos_per.nom_apell, datos_per.fech_ven, datos_red.dir_mac from datos_red, datos_per "
             "where datos_per.coduser=datos_red.coduser and datos_red.dir_mac like  '%"+l+"%'")

    cursor.execute(query)
    

    for (coduser, nom_apell, fech_ven, dir_mac) in cursor:
        print("{} \t {} \t \t {} \t mac {}".format(coduser, nom_apell, fech_ven, dir_mac, l[2], l[3]))



308 	 Jesus Marquina 	 	 2021-04-19 	 mac bc:ff:eb:2c:e8:fc
307 	 Israel Villareal 	 	 2021-05-11 	 mac 9c:a9:e4:f5:d4:89
439 	 Keily Cadenas 	 	 2021-05-10 	 mac 9c:2a:83:75:e8:79
313 	 Jesus AvendaÃ±o 	 	 2021-05-11 	 mac b8:ae:ed:40:60:9b
313 	 Jesus AvendaÃ±o 	 	 2021-05-11 	 mac 50:c8:e5:c7:34:8b


In [18]:
cursor.close()
cnx.close()

In [19]:
q = requests.get('http://192.168.35.73', auth=('castillo', user_pasw))
q

ConnectionError: HTTPConnectionPool(host='192.168.35.73', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fbc9026d070>: Failed to establish a new connection: [Errno 113] No route to host'))